# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city = "./output/final_city.csv"
city_df = pd.read_csv(city)


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

In [4]:
figure_layout = {'width': '500px','height': '400px','border': '1px solid black','padding': '1px'}

fig = gmaps.figure(zoom_level = 1, center = (0,0), layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='500px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
city_df = city_df[city_df['Max Temp'] < 30]
city_df = city_df[city_df['Max Temp'] > 20]
city_df = city_df[city_df['Wind Speed'] < 10]
city_df = city_df[city_df['Cloudiness'] < 50]

city_df = city_df.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = city_df

hotel_df["Hotel Name"] = ""

target_type = "lodging"
radius = 5000

params = {"types": target_type,"radius": radius,"key": g_key}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    
    response_json = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
    except:
        print(f"{row['City']} not found")

hotel_df.head()

east london not found
avarua not found
kapaa not found
boa vista not found
atuona not found
kalmunai not found
arraial do cabo not found
vaini not found
kulhudhuffushi not found
salalah not found
wahiawa not found
kloulklubed not found
sao filipe not found
sabang not found
najran not found
port blair not found
san andres not found
pochutla not found
lazaro cardenas not found
faanui not found
ati not found
cabo san lucas not found
batticaloa not found
san patricio not found
hithadhoo not found
aparecida do taboado not found
serenje not found
gao not found
ambatofinandrahana not found
porto velho not found
acapulco not found
mwingi not found
port elizabeth not found
mercedes not found
cunduacan not found
takoradi not found
tessalit not found
vila velha not found
amambai not found
pointe michel not found
aquidauana not found
pak phanang not found
bara not found
adre not found
boyolangu not found
port lavaca not found
bambanglipuro not found
nabire not found
balikpapan not found
sal rei no

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,east london,-33.0153,27.9116,29.52,54,0,4.63,ZA,1640596015,
1,1,avarua,-21.2078,-159.7750,23.03,83,46,1.54,CK,1640596016,
3,3,kapaa,22.0752,-159.3190,23.18,73,20,0.45,US,1640596017,
18,18,boa vista,2.8197,-60.6733,24.99,78,0,4.12,BR,1640596030,
22,22,atuona,-9.8000,-139.0333,25.91,79,24,6.96,PF,1640596033,


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()])
fig.add_layer(markers)
fig

# Display figure
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))